In [2]:
import pandas as pd


## Les inn data
Les inn to datasett (frå Canvas web_logs via rutina `test_Timer_ny_history.py`): eitt med *alle* data, og eitt med berre dei som skal til Azure

In [3]:
data = pd.read_csv("ny_history.csv", dtype=str)
alledata = pd.read_csv('ny_history_alle.csv', dtype=str)

Plukk ut dei som har "external_tool" i URL-en:

In [3]:
eksterne = data[data['value.url'].str.contains("external_tool")][['value.user_id','value.url']]
alle_eksterne = pd.DataFrame(alledata[alledata['value.url'].str.contains("external_tool")][['value.user_id','value.url']])

Definer ein funksjon som stripper URL for innleiande svada:

In [4]:
def ta_vekk_emne_gruppe(rekke):
    posisjon = [i for i, char in enumerate(rekke['value.url']) if char == '/']
    if len(posisjon) < 4:
        return rekke['value.url']
    else:
        return rekke['value.url'][posisjon[3]+1:]

Og bruk denne på dei to datasetta:

In [5]:
eksterne['ekstern'] = eksterne.apply(ta_vekk_emne_gruppe, axis=1)
alle_eksterne['ekstern'] = alle_eksterne.apply(ta_vekk_emne_gruppe, axis=1)

Definer ein funksjon som klassifiserer eksterne url-er.

In [6]:
def finn_ekstern(ekstern):
    if ekstern.startswith('1?'):
        return 1
    elif ekstern == "3" or ekstern.startswith('3?') or ekstern.startswith('3/resource') or ekstern.startswith('3/launch') or 'office365' in ekstern:
        return 3
    elif ekstern.startswith('24/resource_selection'):
        return 24
    elif ekstern.startswith('55'):
        return 55
    elif ekstern.startswith('64'):
        return 64
    elif ekstern == "193":
        return 193
    elif ekstern.startswith('332') or 'quiz-lti-dub' in ekstern:
        return 332
    elif ekstern.startswith('355'):
        return 355
    elif 'h5p' in ekstern or ekstern.startswith('534'):
        return 534
    elif ekstern == '540':
        return 540
    elif ekstern.startswith('sessionless_launch?') or ekstern.startswith('1247'):
        return 1247
    elif ekstern == '1250':
        return 1250
    elif ekstern == '1252':
        return 1252
    elif ekstern == '1431':
        return 1431
    elif 'panopto' in ekstern or ekstern.startswith('mediasite.usn.no') or ekstern.startswith('1650'):
        return 1650
    else:
        return 999

Og skriv desse til fil (slik at vi kan sjekke dei i Excel):

In [7]:
eksterne['ekstern_kode'] = eksterne['ekstern'].apply(finn_ekstern)
eksterne.to_csv('eksterne.csv')
alle_eksterne['ekstern_kode'] = alle_eksterne['ekstern'].apply(finn_ekstern)
alle_eksterne.to_csv('alle_eksterne.csv')

## Andre ting å leite etter: `api`

Vi må også sjå litt på alle linjene som har /api/ i url-en. Det er ganske mange:

In [8]:
api = alledata[alledata['value.url'].str.contains("/api/")][['key.id', 'value.url', 'value.user_id', 'value.developer_key_id', 'value.web_application_controller', 'value.web_application_action', 'value.web_application_context_type', 'value.web_application_context_id']]

In [9]:
print(f"{len(api)/len(alledata)*100:.1f} % av alle kall har 'api' i url ({len(api)} av {len(alledata)})")

78.8 % av alle kall har 'api' i url (2480123 av 3146884)


Eg ser på eit par linjer:

In [20]:
api.head(5)

,key.id,value.url,value.user_id,value.developer_key_id,value.web_application_controller,value.web_application_action,value.web_application_context_type,value.web_application_context_id
0,ecc755d4-ee45-4782-bcab-1bef68505c14,/api/v1/courses/28643?query_string_redacted,1703,170000000000016,courses,show,Course,28643
1,481b2b62-db4d-49c2-98ed-513d4d6afc23,/api/v1/courses/28933/module_item_sequence?ass...,NaN,NaN,context_module_items_api,item_sequence,Course,28933
2,1d2b219c-e4a5-4472-b276-d68fe9b8ffcc,/api/v1/courses/28154/module_item_sequence?ass...,NaN,NaN,context_module_items_api,item_sequence,Course,28154
3,2ea7363c-746e-43ca-8da4-ae7817975be8,/api/v1/courses/30939?include%5B%5D=banner_ima...,103378,170000000000004,courses,show,Course,30939
4,2b289680-fce4-48f1-aa42-f715b94670aa,/api/v1/courses/27928?query_string_redacted,1703,170000000000016,courses,show,Course,27928


In [11]:
api.iloc[0]['value.url']

'/api/v1/courses/28643?query_string_redacted'

Koden `/api/v1/users/self/todo?include%5B%5D=ungraded_quizzes&per_page=100&no_verifiers=1` henter ut alle "todo"-tinga til ein brukar. Denne vises på dashbordet, så dette ser ut som noko brukaren ikkje bevisst har henta sjølv.

In [12]:
api.iloc[1]['value.url']

'/api/v1/courses/28933/module_item_sequence?asset_type=Assignment&asset_id=83339&frame_external_urls=true'

Koden `/api/v1/courses/28145/module_item_sequence?asset_type=ModuleItem&asset_id=890916&frame_external_urls=true` er eg usikker på kva gjer; det vil sei: den lister opp modulene i eit kurs, men litt usikker på når den vert aktivert. Men den har ikkje brukar registrert, så eg treng eigentleg ikkje sjå på den.

Neste steg: eg sjekker kor mange rader som har `user_id`, kor mange som har `developer_key_id`, og ser om eg ser eit mønster:

In [13]:
user = api[api['value.user_id'].notna()]
developer = api[api['value.developer_key_id'].notna()]
print(f"Det er {len(user)} brukere og {len(developer)} developer.")

Det er 1657413 brukere og 810140 developer.


## `history`
Ok, då må eg sjå litt på dei to kvar for seg. Men ein annan ting: dersom eg ser på *slutten* av datasettet er det veldig mange linjer med "history", og dette er garantert AVO-skriptet. Brukaren er forøvrig "Aasmund Kvamme Administrator" ... Eg ser kor mange det er:

In [14]:
historie = api[api['value.url'].str.contains("/history")]
print(f"Det er {len(historie)} kall med /history i url") # len(historie)

Det er 3005 kall med /history i url


Det var ikkje så mange? "history" burde jo vore sjekka for *alle* studentane? 

## `developer_key_id`
Så ser eg på alle som har `developer_key_id`:

In [15]:
developer['value.developer_key_id'].value_counts()

value.developer_key_id
170000000000004    488997
170000000000016    236946
170000000000044     64686
114                 15585
170000000000463      3046
170000000000465       669
170000000000754        87
170000000000372        66
170000000000485        32
170000000000384        16
170000000000256        10
Name: count, dtype: int64

Her er det noko interessant: kven er desse ulike "developers"?

Statusoppdatering: eg prøver å lese CD2-tabellen `developer_keys`, men den er tom. Og ingen av desse nummera er lista  i `https://hvl.instructure.com/accounts/1/developer_keys`.

## `user_id`
Neste forsøk: kan eg sjå på alle rader med angitt `user_id`?

In [5]:
brukar = alledata[alledata['value.user_id'].notna()]
print(f"Det er {len(brukar)} rader med kjend brukar")

Det er 2086120 rader med kjend brukar


OK, og så ser eg på ein del av desse:

In [7]:
brukar.tail(5)

,key.id,value.timestamp,value.user_id,value.real_user_id,value.course_id,value.quiz_id,value.discussion_id,value.conversation_id,value.assignment_id,value.url,...,value.web_application_controller,value.web_application_action,value.web_application_context_type,value.web_application_context_id,value.session_id,value.developer_key_id,value.participated,value.user_agent,meta.ts,meta.action
3146879,d5db3658-78dc-4e12-9110-eac5de6eab94,2025-01-31T04:05:05.566Z,1703,NaN,29217,NaN,NaN,NaN,NaN,/api/v1/courses/29217?query_string_redacted,...,courses,show,Course,29217,7963b4b2d159ac7d3ef92e8bbf5881fc,170000000000016,false,NaN,2025-01-31T04:48:09.920Z,U
3146880,d5105b61-3a27-418d-916d-3f14032e94ee,2025-01-31T04:05:05.641Z,104682,NaN,30145,NaN,NaN,NaN,84572,/courses/30145/assignments/84572,...,assignments,show,Course,30145,02fe832592f6495e5673a303e7588959,NaN,false,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2025-01-31T04:48:09.920Z,U
3146881,f67a8b9a-e94d-44a7-ae06-9121d355cc96,2025-01-31T04:05:06.538Z,104682,NaN,NaN,NaN,NaN,NaN,NaN,/api/graphql,...,graphql,execute,NaN,NaN,02fe832592f6495e5673a303e7588959,NaN,false,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2025-01-31T04:48:09.920Z,U
3146882,a27f0d46-4db7-439a-9ab7-ae04b4a2dfb3,2025-01-31T04:05:06.909Z,104682,NaN,30145,NaN,NaN,NaN,NaN,/api/lti/courses/30145/jwt_token?tool_launch_u...,...,external_tools,jwt_token,Course,30145,02fe832592f6495e5673a303e7588959,NaN,false,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2025-01-31T04:48:09.920Z,U
3146883,4e90801c-3030-45cc-b48b-b52b95a18921,2025-01-31T04:05:07.964Z,2916,NaN,NaN,NaN,NaN,NaN,NaN,/api/v1/users/73690/history,...,history,index,NaN,NaN,5135f4a19c5228851206e30ba6dd7db5,170000000000016,false,python-requests/2.32.3,2025-01-31T04:48:09.920Z,U


Det eg kan gjere er å sjå kor mange unike brukarar som er registrert, og kor mange rader det er for kvar brukar:

In [54]:
antal_brukarar = alledata['value.user_id'].value_counts()
antal_brukarar.to_csv("antal_brukarar.csv", index=True)

Så kan eg liste alle opplysingar om ein enkelt brukar (av dei som er "ekte" brukarar, altså ikkje slike som meg og NOAS-brukaren):

In [55]:
brukar_6799= alledata[alledata['value.user_id']=="6799"]

### Ulike `controller` og `action`
Eg ser på dei ulike `controller` og `action` i `kjend_brukar`-datasettet:

In [8]:
kontroller = brukar['value.web_application_controller'].value_counts()

In [9]:
action = brukar['value.web_application_action'].value_counts()

In [14]:
kontekst = brukar['value.web_application_context_type'].value_counts()
kontekst

value.web_application_context_type
Course                944187
User                   96315
Account                46762
UserProfile            10562
Group                   5217
CourseSection           4596
AssessmentQuestion        25
Name: count, dtype: int64

In [17]:
kontekst_brukar = brukar[brukar['value.web_application_context_type']=="User"]

In [19]:
kontekst_brukar.head(10)

,key.id,value.timestamp,value.user_id,value.real_user_id,value.course_id,value.quiz_id,value.discussion_id,value.conversation_id,value.assignment_id,value.url,...,value.web_application_controller,value.web_application_action,value.web_application_context_type,value.web_application_context_id,value.session_id,value.developer_key_id,value.participated,value.user_agent,meta.ts,meta.action
25,a73fc6ca-e419-40b5-824b-ee6e816c283d,2025-01-30T11:00:16.123Z,39774,NaN,NaN,NaN,NaN,NaN,NaN,/api/v1/calendar_events?context_codes[]=course...,...,calendar_events_api,index,User,39774,c4f5a164cb56cefa82c8195f5766bc5f,170000000000016,false,NaN,2025-01-30T12:50:47.772Z,U
62,b42a3a52-af98-4599-817a-a7ba6c603e4d,2025-01-30T11:00:19.287Z,93173,NaN,NaN,NaN,NaN,NaN,NaN,/api/v1/users/self/enrollments?per_page=100&st...,...,enrollments_api,index,User,93173,55c7b617dfdb785ece15fa2ddadf0793,170000000000004,false,iCanvas/7.11.1 (24341) iPhone/iOS 18.2.1,2025-01-30T12:50:47.772Z,U
130,5aab4aab-88dc-4ada-9fe3-49ca672febfc,2025-01-30T11:00:27.571Z,44907,NaN,NaN,NaN,NaN,NaN,NaN,/?login_success=1,...,users,user_dashboard,User,44907,6e37cf64412ca40da0a1887ea3b97ce1,NaN,false,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2025-01-30T12:50:47.772Z,U
133,462df6c2-1445-4844-99bc-a223e1ef3969,2025-01-30T11:00:27.764Z,92409,NaN,NaN,NaN,NaN,NaN,NaN,/,...,users,user_dashboard,User,92409,8eb9b88e60dba688e859531578841a61,NaN,false,Mozilla/5.0 (iPhone; CPU iPhone OS 18_1_1 like...,2025-01-30T12:50:47.772Z,U
167,fa609f28-e51b-4964-9e42-d7476d1b86df,2025-01-30T11:00:30.059Z,93007,NaN,NaN,NaN,NaN,NaN,NaN,/,...,users,user_dashboard,User,93007,0e69973de881acb53404330b65d64f0e,NaN,false,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2025-01-30T12:50:47.772Z,U
176,69dc18b9-33bc-4257-ac5c-724f85812856,2025-01-30T11:00:30.360Z,7883,NaN,NaN,NaN,NaN,NaN,NaN,/,...,users,user_dashboard,User,7883,377ed3d55475dee440c0940479ed5c2e,NaN,false,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2025-01-30T12:50:47.772Z,U
183,e30e6b62-a21a-49b3-85e9-2c9638364d28,2025-01-30T11:00:30.433Z,34333,NaN,NaN,NaN,NaN,NaN,NaN,/files/1637775/download?download_frd=1&verifie...,...,files,show,User,34333,5278c8e4541d6e60fa1ed1cda3a53d9c,NaN,false,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2025-01-30T12:50:47.772Z,U
241,f6100de9-4e5e-4788-96af-dbae23067775,2025-01-30T11:00:32.142Z,39774,NaN,NaN,NaN,NaN,NaN,NaN,/api/v1/calendar_events?context_codes[]=course...,...,calendar_events_api,index,User,39774,50c5274a22e9abdc4630a3f1bd5f9228,170000000000016,false,NaN,2025-01-30T12:50:47.772Z,U
243,90b5d810-83cb-4385-ae6f-ea2a427a13fd,2025-01-30T11:00:32.154Z,34333,NaN,NaN,NaN,NaN,NaN,NaN,/api/v1/calendar_events?per_page=50&type=assig...,...,calendar_events_api,index,User,34333,5278c8e4541d6e60fa1ed1cda3a53d9c,NaN,false,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2025-01-30T12:50:47.772Z,U
257,f5e724c9-7fc3-4b7d-b466-053235bfa7df,2025-01-30T11:00:32.564Z,80525,NaN,NaN,NaN,NaN,NaN,NaN,/files/2024323/download?download_frd=1&verifie...,...,files,show,User,80525,6bc0f5d08d197910c8f139560d563dd9,NaN,false,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2025-01-30T12:50:47.772Z,U


In [21]:
kontekst_brukar['value.web_application_action'].value_counts()

value.web_application_action
user_dashboard        32687
index                 32338
public_feed           13516
show                   6970
get_data               6862
enabled_features       3108
resolve_path            212
create_pending          196
api_quota               126
create                   84
create_file              76
manageable_courses       51
api_show                 41
react_files              20
api_index                13
update                    6
set_data                  5
new                       2
api_update                2
Name: count, dtype: int64

In [22]:
kontekst_brukar['value.web_application_controller'].value_counts()

value.web_application_controller
users                  32816
calendar_events_api    30263
enrollments_api        13786
custom_data             6867
files                   6275
feature_flags           3196
tabs                    1724
calendars                921
folders                  363
file_previews             99
calendar_events            4
tokens                     1
Name: count, dtype: int64

In [24]:
filer = kontekst_brukar[kontekst_brukar['value.web_application_controller']=="files"]
filer.to_csv("files.csv", index=False)

In [25]:
filer['value.web_application_action'].value_counts()

value.web_application_action
show              5805
create_pending     196
api_quota          126
index               72
api_show            41
react_files         20
api_index           13
api_update           2
Name: count, dtype: int64

## Krysstabell
Kunne det vore interesant å sjå på ein krysstabell mellom `value.web_application_controller` og `value.web_application_action`?

In [10]:
filtrert = brukar[(brukar['value.web_application_controller'].isin(['courses', 'users', 'files'])) & (brukar['value.web_application_action'].isin(['index', 'show']))]

In [11]:
pivot_tabell = pd.pivot_table(
    filtrert,
    index=['value.web_application_controller'],
    columns=['value.web_application_action'],
    values=['value.user_id'],
    aggfunc='count'
)

In [12]:
print(pivot_tabell)

                                 value.user_id          
value.web_application_action             index      show
value.web_application_controller                        
courses                                78859.0  177401.0
files                                   1529.0   55421.0
users                                      NaN     637.0


Det ga ikkje så mykje interessant, trur eg. Men det er jo litt av di eg ikkje er sikker på kva dei ulike kolonnene eigentleg betyr. Og eg er jo på jakt etter "reelle" tastetrykk frå studentar, ikkje automatiserte kommandoar generert av Canvas sjølv.



## `session_id`
Kva med å sjå på ein komplett "session" for ein student, og sjå om eg kan finne noko der?


In [29]:
sesjon = alledata[alledata['value.session_id']=="07de262b8a2c8c27ad942fa89f40c11b"]
sesjon.to_csv('sesjon.csv', index=False)

In [30]:
sesjon

,key.id,value.timestamp,value.user_id,value.real_user_id,value.course_id,value.quiz_id,value.discussion_id,value.conversation_id,value.assignment_id,value.url,...,value.web_application_controller,value.web_application_action,value.web_application_context_type,value.web_application_context_id,value.session_id,value.developer_key_id,value.participated,value.user_agent,meta.ts,meta.action
43070,c0c568b1-3753-4947-9d5e-903a4dda855f,2025-01-30T11:08:59.354Z,110005,NaN,NaN,NaN,NaN,NaN,NaN,/api/v1/conversations/unread_count,...,conversations,unread_count,NaN,NaN,07de262b8a2c8c27ad942fa89f40c11b,NaN,false,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2025-01-30T12:50:47.772Z,U
43239,1f017c44-c5c3-4028-af96-ab0ed8506939,2025-01-30T11:09:00.477Z,110005,NaN,29766,NaN,NaN,NaN,NaN,/courses/29766/external_tools/55,...,external_tools,show,Course,29766,07de262b8a2c8c27ad942fa89f40c11b,NaN,false,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2025-01-30T12:50:47.772Z,U
43285,bdbc02c6-5527-4417-96fa-c55e0a12c2e0,2025-01-30T11:09:00.983Z,110005,NaN,29766,NaN,NaN,NaN,NaN,/api/lti/courses/29766/jwt_token?tool_launch_u...,...,external_tools,jwt_token,Course,29766,07de262b8a2c8c27ad942fa89f40c11b,NaN,false,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2025-01-30T12:50:47.772Z,U
43308,fc94e50c-6682-4689-9027-3583abb62c5c,2025-01-30T11:09:01.171Z,110005,NaN,NaN,NaN,NaN,NaN,NaN,/api/lti/authorize?lti_message_hint=eyJ0eXAiOi...,...,lti/ims/authentication,authorize,Course,29766,07de262b8a2c8c27ad942fa89f40c11b,NaN,false,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2025-01-30T12:50:47.772Z,U
107791,704ffe53-5cf5-4040-9d22-352b0ed2bb12,2025-01-30T11:18:47.270Z,110005,NaN,NaN,NaN,NaN,NaN,NaN,/api/v1/conversations/unread_count,...,conversations,unread_count,NaN,NaN,07de262b8a2c8c27ad942fa89f40c11b,NaN,false,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2025-01-30T12:50:47.772Z,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231159,dad8466e-4021-4e96-95aa-d5d48820c26e,2025-01-30T10:38:19.140Z,110005,NaN,NaN,NaN,NaN,NaN,NaN,/api/v1/files/3155755,...,files,api_show,NaN,NaN,07de262b8a2c8c27ad942fa89f40c11b,NaN,false,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2025-01-30T11:50:55.459Z,U
1907042,64887175-2965-4181-989d-47ec40ffcbe9,2025-01-30T13:01:04.647Z,110005,NaN,29766,NaN,NaN,NaN,NaN,/api/v1/courses/29766/ping,...,courses,ping,Course,29766,07de262b8a2c8c27ad942fa89f40c11b,NaN,false,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2025-01-30T13:51:08.536Z,U
1907043,5fb0bfcc-e95d-4bf8-a5d1-79050ff993ba,2025-01-30T13:01:04.648Z,110005,NaN,NaN,NaN,NaN,NaN,NaN,/api/v1/conversations/unread_count,...,conversations,unread_count,NaN,NaN,07de262b8a2c8c27ad942fa89f40c11b,NaN,false,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2025-01-30T13:51:08.536Z,U
1907486,2382710d-8fb6-4f2b-9632-0967636aa17b,2025-01-30T13:01:12.590Z,110005,NaN,29766,NaN,NaN,NaN,NaN,/api/lti/courses/29766/jwt_token?tool_launch_u...,...,external_tools,jwt_token,Course,29766,07de262b8a2c8c27ad942fa89f40c11b,NaN,false,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2025-01-30T13:51:08.536Z,U


In [49]:
antal_brukarar = alledata['value.user_id'].value_counts()
antal_brukarar.to_csv("antal_brukarar.csv", index=True)

In [50]:
brukar_6799= alledata[alledata['value.user_id']=="6799"]

In [51]:
brukar_6799.to_csv("brukar_6799.csv", index=False)